# Developing Word Embeddings

Rather than use pre-trained embeddings (as we did in the baseline_deep_dive notebook), we can train word embeddings using our own dataset. In this notebook, we demonstrate the training process for producing word embeddings using the word2vec, GloVe, and fastText models. We'll utilize the STS Benchmark dataset for this task. 

# Table of Contents
* [Data Loading and Preprocessing](#Load-and-Preprocess-Data)
* [Word2Vec](#Word2Vec)
* [fastText](#fastText)
* [GloVe](#GloVe)
* [Concluding Remarks](#Concluding-Remarks)

In [4]:
import gensim
import sys
import os

## Load and Preprocess Data

In [5]:
# Set the environment path
sys.path.append("../../../") 
# Set the path for where your datasets are located
BASE_DATA_PATH = "../../../data" 
# Location to save embeddings
SAVE_FILES_PATH = BASE_DATA_PATH + "/trained_word_embeddings/"

if not os.path.exists(SAVE_FILES_PATH):
    os.makedirs(SAVE_FILES_PATH)
    
from utils_nlp.dataset.preprocess import (
    to_lowercase,
    to_spacy_tokens,
    rm_spacy_stopwords,
)
from utils_nlp.dataset import stsbenchmark

In [6]:
# Produce a pandas dataframe for the training set
sts_train = stsbenchmark.load_pandas_df(BASE_DATA_PATH, file_split="train")

#### Training set preprocessing

In [7]:
# Convert all text to lowercase
df_low = to_lowercase(sts_train)  
# Tokenize text
sts_tokenize = to_spacy_tokens(df_low) 
# Tokenize with removal of stopwords
sts_train_stop = rm_spacy_stopwords(sts_tokenize) 

In [8]:
# Append together the two sentence columns to get a list of all tokenized sentences
all_sentences =  sts_train[["sentence1_tokens_rm_stopwords", "sentence2_tokens_rm_stopwords"]]
sentences = all_sentences.values.flatten().tolist()

In [9]:
sentences[:10]

0                             [plane, taking, .]
1                [man, playing, large, flute, .]
2    [man, spreading, shreded, cheese, pizza, .]
3                       [men, playing, chess, .]
4                       [man, playing, cello, .]
5                             [men, fighting, .]
6                              [man, smoking, .]
7                       [man, playing, piano, .]
8             [man, playing, guitar, singing, .]
9            [person, throwing, cat, ceiling, .]
dtype: object

## Word2Vec

Word2vec is a predictive model for learning word embeddings from text. Word embeddings are learned such that words that share common contexts in the corpus will be close together in the vector space. There are two different model architectures that can be used to produce word2vec embeddings: continuous bag-of-words (CBOW) or continuous skip-gram. The former uses a window of surrounding words (the "context") to predict the current word and the latter uses the current word to predict the surrounding context words. See this [tutorial](https://www.guru99.com/word-embedding-word2vec.html#3) on word2vec for more detailed background on the model.

The gensim Word2Vec model has many different parameters (see [here](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec)) but the ones that are useful to know about are:  
- size: length of the word embedding/vector (defaults to 100)
- window: maximum distance between the word being predicted and the current word (defaults to 5)
- min_count: ignores all words that have a frequency lower than this value (defaults to 5)
- workers: number of worker threads used to train the model (defaults to 3)
- sg: training algorithm; 1 for skip-gram and 0 for CBOW (defaults to 0)

In [10]:
from gensim.models import Word2Vec

word2vec_model = Word2Vec(sentences, size=100, window=5, min_count=5, workers=3, sg=0)

Now that the model is trained we can:

1. Query for the word embeddings of a given word. 
2. Inspect the model vocabulary
3. Save the word embeddings

In [11]:
# 1. Let's see the word embedding for "apple" by accessing the "wv" attribute and passing in "apple" as the key.
print("Embedding for apple:", word2vec_model.wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words
print("\nFirst 30 vocabulary words:", list(word2vec_model.wv.vocab)[:20])

# 3. Save the word embeddings. We can save as binary format (to save space) or ASCII format
word2vec_model.wv.save_word2vec_format(SAVE_FILES_PATH+"word2vec_model", binary=True)  # binary format
word2vec_model.wv.save_word2vec_format(SAVE_FILES_PATH+"word2vec_model", binary=False)  # ASCII format

Embedding for apple: [ 0.01967091 -0.02060411  0.06045751  0.18435013  0.06008527 -0.03161449
  0.03215135  0.20443363 -0.11477666 -0.09098607  0.15125929 -0.12137079
 -0.03533404  0.07740553 -0.0655808  -0.21782503  0.11100145  0.16948767
  0.03154546 -0.0302508  -0.16382743 -0.06409733 -0.07107548  0.01911855
  0.07808343 -0.00865842 -0.11687949 -0.0813501  -0.07051189 -0.13114788
  0.06511062  0.09063648  0.00589781  0.14914463  0.1419142  -0.3039675
 -0.02749436 -0.14556152  0.19307302 -0.0151168   0.05703858  0.07629551
  0.10458219  0.08367997  0.05312656 -0.05101583 -0.03869054 -0.06782712
 -0.12123356 -0.2769428   0.2597927  -0.07549391 -0.20704848 -0.00398967
 -0.03525157  0.19550665 -0.11829475 -0.16258994  0.09406583 -0.05389677
 -0.11935548  0.11754951  0.00271186  0.17870599 -0.06797015  0.05897843
  0.07160962  0.01180044 -0.16571392  0.10249291  0.02082502  0.09404808
 -0.01148031 -0.04788112 -0.14149055 -0.21472694 -0.00461547 -0.11108381
  0.06521665  0.05437631 -0.043

## fastText

fastText is an unsupervised algorithm created by Facebook Research for efficiently learning word embeddings. fastText is significantly different than word2vec or GloVe in that these two algorithms treat each word as the smallest possible unit to find an embedding for. Conversely, fastText assumes that words are formed by an n-gram of characters (i.e. 2-grams of the word "language" would be {la, an, ng, gu, ua, ag, ge}). The embedding for a word is then composed of the sum of these character n-grams. This has advantages when finding word embeddings for rare words and words not present in the dictionary, as these words can still be broken down into character n-grams. Typically, for smaller datasets, fastText performs better than word2vec or GloVe. See this [tutorial](https://fasttext.cc/docs/en/unsupervised-tutorial.html) on fastText for more detail.

The gensim fastText model has many different parameters (see [here](https://radimrehurek.com/gensim/models/fasttext.html#gensim.models.fasttext.FastText)) but the ones that are useful to know about are:  
- size: length of the word embedding/vector (defaults to 100)
- window: maximum distance between the word being predicted and the current word (defaults to 5)
- min_count: ignores all words that have a frequency lower than this value (defaults to 5)
- workers: number of worker threads used to train the model (defaults to 3)
- sg: training algorithm- 1 for skip-gram and 0 for CBOW (defaults to 0)
- iter: number of epochs (defaults to 5)


In [12]:
from gensim.models.fasttext import FastText

fastText_model = FastText(size=100, window=5, min_count=5, sentences=sentences, iter=5)

We can utilize the same attributes as we saw above for word2vec due to them both originating from the gensim package

In [13]:
# 1. Let's see the word embedding for "apple" by accessing the "wv" attribute and passing in "apple" as the key.
print("Embedding for apple:", fastText_model.wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words
print("\nFirst 30 vocabulary words:", list(fastText_model.wv.vocab)[:20])

# 3. Save the word embeddings. We can save as binary format (to save space) or ASCII format
fastText_model.wv.save_word2vec_format(SAVE_FILES_PATH+"fastText_model", binary=True)  # binary format
fastText_model.wv.save_word2vec_format(SAVE_FILES_PATH+"fastText_model", binary=False)  # ASCII format

Embedding for apple: [ 0.13790123  0.01106163  0.592905   -0.12064535 -0.24870746  0.23486118
 -0.17894644 -0.02935349 -0.57857525  0.24043512  0.1748657   0.24043098
  0.56225604  0.35615742  0.02726375  0.5418794  -0.4584483  -0.26613978
  0.10734484  0.14210673  0.279661    0.23946704 -0.07208388 -0.16432633
  0.19801465 -0.17888714 -0.17770861 -0.06616489  0.45454445 -0.03997259
  0.16976304 -0.18167289  0.3837792   0.32586682  0.4405366  -0.23304892
 -0.05752061  0.90343505 -0.06364796  0.28235433 -0.2506317   0.0584392
 -0.34491786  0.09534133 -0.11522547 -0.03102012 -0.14524432 -0.293817
 -0.18027557  0.11820582 -0.2977447   0.18128611  0.07384928  0.30933177
  0.18112482 -0.31221172  0.07127812  0.41098395  0.1478829  -0.09991158
  0.30086008  0.04828371  0.18450771  0.15665273  0.12014977 -0.57937026
  0.5833518   0.00775435 -0.09244794 -0.1508434   0.06862748 -0.12566844
  0.07151426  0.39914075  0.04042512  0.02652129  0.14254047  0.18192361
 -0.1524721   0.1218605   0.40592

## GloVe

GloVe is an unsupervised algorithm for obtaining word embeddings created by the Stanford NLP group. Training occurs on word-word co-occurrence statistics with the objective of learning word embeddings such that the dot product of two words' embeddings is equal to the words' probability of co-occurrence. See this [tutorial](https://nlp.stanford.edu/projects/glove/) on GloVe for more detailed background on the model. 

Gensim doesn't have an implementation of the GloVe model, so we suggest getting the code directly from the Stanford NLP [repo](https://github.com/stanfordnlp/GloVe). Run the following commands to clone the repo and then make. Clone the repo in the same location as this notebook! Otherwise, the paths below will need to be modified.  

    git clone http://github.com/stanfordnlp/glove    
    cd glove && make  

### Train GloVe vectors

Training GloVe embeddings requires some data prep and then 4 steps (also documented in the original Stanford NLP repo [here](https://github.com/stanfordnlp/GloVe/tree/master/src)).

**Step 0: Prepare Data**
   
In order to train our GloVe vectors, we first need to save our corpus as a text file with all words separated by 1+ spaces or tabs. Each document/sentence is separated by a new line character.

In [14]:
# Save our corpus as tokens delimited by spaces with new line characters in between sentences
with open(BASE_DATA_PATH+'/clean/stsbenchmark/training-corpus-cleaned.txt', 'w', encoding='utf8') as file:
    for sent in sentences:
        file.write(" ".join(sent) + "\n")

**Step 1: Build Vocabulary**

Run the vocab_count executable. There are 3 optional parameters:
1. min-count: lower limit on how many times a word must appear in dataset. Otherwise the word is discarded from our vocabulary.
2. max-vocab: upper bound on the number of vocabulary words to keep
3. verbose: 0, 1, or 2 (default)

Then provide the path to the text file we created in Step 0 followed by a file path that we'll save the vocabulary to 

In [15]:
!"glove/build/vocab_count" -min-count 5 -verbose 2 <"../../../data/clean/stsbenchmark/training-corpus-cleaned.txt"> "../../../data/trained_word_embeddings/vocab.txt"

BUILDING VOCABULARY
Processed 0 tokens.Processed 84997 tokens.
Counted 11716 unique words.
Truncating vocabulary at min count 5.
Using vocabulary of size 2943.



**Step 2: Construct Word Co-occurrence Statistics**

Run the cooccur executable. There are many optional parameters, but we list the top ones here:
1. symmetric: 0 for only looking at left context, 1 (default) for looking at both left and right context
2. window-size: number of context words to use (default 15)
3. verbose: 0, 1, or 2 (default)
4. vocab-file: path/name of the vocabulary file created in Step 1
5. memory: soft limit for memory consumption, default 4
6. max-product: limit the size of dense co-occurrence array by specifying the max product (integer) of the frequency counts of the two co-occurring words

Then provide the path to the text file we created in Step 0 followed by a file path that we'll save the co-occurrences to

In [16]:
!"glove/build/cooccur" -memory 4 -vocab-file "../../../data/trained_word_embeddings/vocab.txt" -verbose 2 -window-size 15 <"../../../data/clean/stsbenchmark/training-corpus-cleaned.txt"> "../../../data/trained_word_embeddings/cooccurrence.bin"

COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 13752509
overflow length: 38028356
Reading vocab from file "../../../data/trained_word_embeddings/vocab.txt"...loaded 2943 words.
Building lookup table...table contains 8661250 elements.
Processing token: 0Processed 84997 tokens.
Writing cooccurrences to disk......2 files in total.
Merging cooccurrence files: processed 0 lines.0 lines.100000 lines.Merging cooccurrence files: processed 187717 lines.



**Step 3: Shuffle the Co-occurrences**

Run the shuffle executable. The parameters are as follows:
1. verbose: 0, 1, or 2 (default)
2. memory: soft limit for memory consumption, default 4
3. array-size: limit to the length of the buffer which stores chunks of data to shuffle before writing to disk

Then provide the path to the co-occurrence file we created in Step 2 followed by a file path that we'll save the shuffled co-occurrences to

In [17]:
!"glove/build/shuffle" -memory 4 -verbose 2 <"../../../data/trained_word_embeddings/cooccurrence.bin"> "../../../data/trained_word_embeddings/cooccurrence.shuf.bin"

SHUFFLING COOCCURRENCES
array size: 255013683
Shuffling by chunks: processed 0 lines.processed 187717 lines.
Wrote 1 temporary file(s).
Merging temp files: processed 0 lines.187717 lines.Merging temp files: processed 187717 lines.



**Step 4: Train GloVe model**

Run the glove executable. There are many parameter options, but the top ones are listed below:
1. verbose: 0, 1, or 2 (default)
2. vector-size: dimension of word embeddings (50 is default)
3. threads: number threads, default 8
4. iter: number of iterations, default 25
5. eta: learning rate, default 0.05
6. binary: whether to save binary format (0: text = default, 1: binary, 2: both)
7. x-max: cutoff for weighting function, default is 100
8. vocab-file: file containing vocabulary as produced in Step 1
9. save-file: filename to save vectors to 
10. input-file: filename with co-occurrences as returned from Step 3

In [18]:
!"glove/build/glove" -save-file "../../../data/trained_word_embeddings/GloVe_vectors" -threads 8 -input-file \
"../../../data/trained_word_embeddings/cooccurrence.shuf.bin" -x-max 10 -iter 15 -vector-size 50 -binary 2 \
-vocab-file "../../../data/trained_word_embeddings/vocab.txt" -verbose 2

TRAINING MODEL
Read 187717 lines.
Initializing parameters...done.
vector size: 50
vocab size: 2943
x_max: 10.000000
alpha: 0.750000
05/01/19 - 11:58.04AM, iter: 001, cost: 0.078324
05/01/19 - 11:58.04AM, iter: 002, cost: 0.072073
05/01/19 - 11:58.04AM, iter: 003, cost: 0.070105
05/01/19 - 11:58.05AM, iter: 004, cost: 0.067233
05/01/19 - 11:58.05AM, iter: 005, cost: 0.063541
05/01/19 - 11:58.05AM, iter: 006, cost: 0.060692
05/01/19 - 11:58.05AM, iter: 007, cost: 0.058141
05/01/19 - 11:58.05AM, iter: 008, cost: 0.056148
05/01/19 - 11:58.05AM, iter: 009, cost: 0.054026
05/01/19 - 11:58.05AM, iter: 010, cost: 0.051821
05/01/19 - 11:58.05AM, iter: 011, cost: 0.049579
05/01/19 - 11:58.05AM, iter: 012, cost: 0.047395
05/01/19 - 11:58.05AM, iter: 013, cost: 0.045237
05/01/19 - 11:58.05AM, iter: 014, cost: 0.043165
05/01/19 - 11:58.05AM, iter: 015, cost: 0.041168


### Inspect Word Vectors

Like we did above for the word2vec and fastText models, let's now inspect our word embeddings

In [19]:
#load in the saved word vectors
glove_wv = {}
with open("../../../data/trained_word_embeddings/GloVe_vectors.txt", encoding='utf-8') as f:
    for line in f:
        split_line = line.split(" ")
        glove_wv[split_line[0]] = [float(i) for i in split_line[1:]]

In [20]:
# 1. Let's see the word embedding for "apple" by passing in "apple" as the key.
print("Embedding for apple:", glove_wv["apple"])

# 2. Inspect the model vocabulary by accessing keys of the "wv.vocab" attribute. We'll print the first 20 words
print("\nFirst 30 vocabulary words:", list(glove_wv.keys())[:20])

Embedding for apple: [0.108406, -0.069786, 0.062411, 0.101886, 0.027323, -0.106115, 0.027733, 0.033604, -0.071702, 0.080405, -0.054496, 0.138576, 0.026596, 0.059057, 0.044539, -0.074442, 0.057427, -0.064205, 0.023717, -0.01441, 0.140524, 0.130299, 0.053629, 0.138093, -0.215051, 0.007554, 0.022354, 0.123382, -0.117929, -0.069709, 0.049555, -0.059443, -0.092295, 0.078103, -0.011445, -0.024891, 0.025378, -0.054191, -0.014498, -0.064865, 0.056489, -0.038174, 0.014945, -0.115485, 0.126237, 0.091793, -0.085583, 0.070163, -0.196238, 0.003365]

First 30 vocabulary words: ['.', ',', 'man', '-', 'woman', "'", 'said', 'dog', '"', 'playing', ':', 'white', 'black', '$', 'killed', 'percent', 'new', 'syria', 'people', 'china']


# Concluding Remarks

In this notebook we have shown how to train word2vec, GloVe, and fastText word embeddings on the STS Benchmark dataset. FastText is typically regarded as the best baseline for word embeddings (see [blog](https://medium.com/huggingface/universal-word-sentence-embeddings-ce48ddc8fc3a)) and is a good place to start when generating word embeddings. Now that we generated word embeddings on our dataset, we could also repeat the baseline_deep_dive notebook using these embeddings (versus the pre-trained ones from the internet). 